In [1]:
# !pip install -U torch transformers accelerate sentence-transformers
# !pip uninstall spellchecker -y
# !pip install pyspellchecker
# !pip install unidecode
# !pip install scikit-learn  # For cosine_similarity and numpy (it's a dependency)
# !pip install sentence-transformers
# !pip install pandas
# !pip install ipywidgets


from sentence_transformers import SentenceTransformer
import numpy as np
import pandas as pd
import datadetox as detox
import importlib

Leitura e padronização de colunas de chaves primarias para concatenação de dfs

In [36]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
path = r'C:\Users\USER\Desktop\GitHub\Brasileirao-Analise\1. Data\Transformed'

#Games 
df_list = []
for i in range(1, 8):
    df = pd.read_excel(path + f'\\0{i}_games.xlsx')
    df_list.append(df)


#fitragem série B conjunto 5
df_list[6] = df_list[6][df_list[6]['Série']=='A']

# mapeamento_colunas chave primária
mapeamento_colunas = {  'data': 'Data','Date': 'Data','data_partida': 'Data',
                        'time_mandante': 'Mandante','mandante': 'Mandante','Home': 'Mandante',
                        'time_visitante': 'Visitante','visitante': 'Visitante','Away': 'Visitante'}
    

df_list = [df.rename(columns=mapeamento_colunas) for df in df_list]

df_list = detox.padronizar_datas(df_list)

for i, df_save in enumerate(df_list):
    df_save.to_excel(path + f'\\0{i+1}_games.xlsx',index=False)


DataFrame 0: Já é datetime64, mantido como está.
DataFrame 1: Já é datetime64, mantido como está.
DataFrame 2: Já é datetime64, mantido como está.
DataFrame 3: Já é datetime64, mantido como está.
DataFrame 4: Já é datetime64, mantido como está.
DataFrame 5: Já é datetime64, mantido como está.
DataFrame 6: Já é datetime64, mantido como está.


Unir todos os DFs com agrupamento por colunas, Data, Mandante e Visitante

In [38]:
#mapear cluber qeu o filtro BERT nao resolve (manualmente)
clubes_map_inicial = {  "America-RN":"América Natal","América-RN":"América Natal","América rn":"América Natal",
                        "Joinvile":"Joinville",
                        "Atlético pr":"Athlético-PR","Atlético-PR":"Athlético-PR", 
                        "Grêmio Prudente":"Barueri","Grêmio prudente":"Barueri","Gremio Prudente":"Barueri"}

for df in df_list:
    df['Mandante'] = df['Mandante'].replace(clubes_map_inicial)
    df['Visitante'] = df['Visitante'].replace(clubes_map_inicial)

# listar clubes Mandantes e visitantes, para agrupar ortograficamente
clubes_unicos = []
for df in df_list:
    unicos_por_bd = pd.unique(np.concatenate([df['Mandante'].values, df['Visitante'].values])).tolist()
    clubes_unicos = list(set(clubes_unicos) | set(unicos_por_bd))

clubes_value_dict = [   "América Mineiro", "América Natal", "Athletico Paranaense", "Atlético Goianiense", "Atlético Mineiro", "Avaí", "Bahia", 
                        "Barueri", "Botafogo", "Red Bull Bragantino", "Brasiliense", "Ceará", 
                        "Chapecoense", "Corinthians", "Coritiba", "Criciúma", "Cruzeiro", "CSA", "Cuiabá", "Figueirense", "Flamengo", "Fluminense", 
                        "Fortaleza","Guarani", "Goiás", "Grêmio", "Grêmio Prudente", "Internacional", "Ipatinga", "Joinville", "Juventude","Mirassol", "Náutico", 
                        "Palmeiras", "Paraná", "Paysandu", "Ponte Preta", "Portuguesa", "Santa Cruz", "Santo André", "Santos", "São Caetano", "São Paulo", 
                        "Sport Recife", "Vasco da Gama", "Vitória"]

modelo = SentenceTransformer('all-MiniLM-L6-v2')
clubes_corrigidos = detox.corrigir_times_semanticamente(clubes_unicos, clubes_value_dict,modelo)
clubes_dict = detox.gerar_dicionario_mapeamento(clubes_corrigidos, limite=0.0)

# corrigindo e agrupando clubes em todos os dataframes
for df in df_list:
    df['Mandante'] = df['Mandante'].replace(clubes_dict)
    df['Visitante'] = df['Visitante'].replace(clubes_dict)

# unindo todos os dfs com merge on=['Data','Mandante','Visitante']
df_merged = pd.DataFrame(columns=['Data','Mandante','Visitante'])
for i,df in enumerate(df_list):
    df_merged = pd.merge(df_merged, df, how='outer', on=['Data','Mandante','Visitante'],suffixes=('', f'_{i}'))


In [39]:
pd.set_option('display.max_rows', None)
import datetime as dt
df = df_merged.copy()

# Criar edições dos campeonatos visto que 2020 foi ate 2021
mask = (df['Data'] > dt.datetime(2020, 8, 1)) & (df['Data'] < dt.datetime(2021, 2, 28))
df.loc[mask, 'edition'] = 2020
df.loc[~mask, 'edition'] = df.loc[~mask, 'Data'].dt.year

df.sort_values(by=['Mandante', 'Visitante','edition','Data'], ascending=[True, True, True, True], inplace=True)
#df.head(10)

In [40]:
importlib.reload(detox)

# Agrupar e aplicar a função
df_ajustado = df.groupby(['Mandante', 'Visitante', 'edition'], group_keys=False).apply(detox.ajustar_datas)

# Aplica o agrupamento e mescla as linhas sem conflito
df_mesclado = (
    df_ajustado.groupby(['Data', 'Mandante', 'Visitante', 'edition'], group_keys=False)
      .apply(detox.mesclar_linhas_sem_conflito)
      .reset_index(drop=True))

df_mesclado = df_mesclado[df_mesclado['edition']<2024]

df_group = df_mesclado.groupby(['Mandante', 'Visitante', 'edition'], group_keys=False).apply(
    lambda group: detox.ajustar_datas_semdiff(group,keep=True))

df_mesclado2 = (
    df_group.groupby(['Data', 'Mandante', 'Visitante', 'edition'], group_keys=False)
      .apply(detox.mesclar_linhas_sem_conflito)
      .reset_index(drop=True))

df_mesclado2.to_excel(r'C:\Users\USER\Desktop\GitHub\Brasileirao-Analise\1. Data\Transformed\Agrupamento_unico.xlsx',index=False)

C:\Users\USER\AppData\Local\Temp\ipykernel_13072\2427877954.py:4: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_ajustado = df.groupby(['Mandante', 'Visitante', 'edition'], group_keys=False).apply(detox.ajustar_datas)
C:\Users\USER\AppData\Local\Temp\ipykernel_13072\2427877954.py:9: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  .apply(detox.mesclar_linhas_sem_conflito)
C:\Users\USER\AppData\Local\Temp\ipykernel_13072\2427877954.py:9: DeprecationWarn

Garantindo partidas por temporada

In [54]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 150)
importlib.reload(detox)

df = pd.read_excel(r'C:\Users\USER\Desktop\GitHub\Brasileirao-Analise\1. Data\Transformed\Agrupamento_unico.xlsx')

#ajustando sao paulo e sao caetano jogo interrompido
df.drop(1007, inplace=True)

# # ajustando jogo do flamengo
# df.loc[2835, 'Data'] = pd.to_datetime('2009-11-07')
# df.drop(2834, inplace=True)

# # ajustando jogo do BOTAFOGO
# df.loc[1969, 'Data'] = pd.to_datetime('2009-07-29')
# df.drop(1968, inplace=True)

# # ajustando jogo da chapecoense
# df.drop(5753, inplace=True)

#2005
df.drop([1137,1137,1207,1231,1257,1284,1307,1310,1328,1341,1375,1379], inplace=True)
df.loc[1138, 'Data'] = pd.to_datetime('2009-10-19')
df.loc[1208, 'Data'] = pd.to_datetime('2009-10-19')
df.loc[1232, 'Data'] = pd.to_datetime('2009-10-19')
df.loc[1258, 'Data'] = pd.to_datetime('2009-10-19')
df.loc[1285, 'Data'] = pd.to_datetime('2009-10-19')
df.loc[1308, 'Data'] = pd.to_datetime('2009-10-19')
df.loc[1311, 'Data'] = pd.to_datetime('2009-10-19')
df.loc[1329, 'Data'] = pd.to_datetime('2009-10-19')
df.loc[1342, 'Data'] = pd.to_datetime('2009-10-19')
df.loc[1376, 'Data'] = pd.to_datetime('2009-10-19')
df.loc[1380, 'Data'] = pd.to_datetime('2009-10-19')

#2006 
df.drop([1695,1697,1724,1726], inplace=True)

#2007 
df.drop([1973,1974,1975], inplace=True)

#2009 
df.drop([2839], inplace=True)

detox.verifica_partidas_edi(df,'edition','Mandante')
detox.verifica_partidas_edi(df,'edition','Visitante')

df.to_excel(r'C:\Users\USER\Desktop\GitHub\Brasileirao-Analise\1. Data\Transformed\Num_partidas.xlsx',index=False)

Edições com número desigual de partidas por time: Series([], Name: n_partidas, dtype: int64)
Edições com número desigual de partidas por time: Series([], Name: n_partidas, dtype: int64)


Meslclando Colunas

In [116]:
importlib.reload(detox)

df = pd.read_excel(r'C:\Users\USER\Desktop\GitHub\Brasileirao-Analise\1. Data\Transformed\Num_partidas.xlsx')

gols_mandante = ["gols_mandante","mandante_Placar","Mandante Placar","mandante_Placar_3","Gol Mandante","placar_mandante"]
gols_visitante = ["gols_visitante","visitante_Placar","Visitante Placar","visitante_Placar_3","Gol Visitante","placar_visitante"]
colunas_hora = ["Horário", "hora", "hora_3", "Hora", "hora_partida"]
arenas_col = ['arena', 'Arena', 'arena_3', 'Estádio', 'estadio', 'estadio_5']
rodada_col = ['rodada','rodata','Rodada','rodata_3','rodada_5','Rodada_6']

for col in colunas_hora:
    df[col] = df[col].str.replace(r'h', ':', regex=True)
    df[col] = pd.to_datetime(df[col], format='%H:%M')

df[gols_mandante].nunique(axis=1,dropna=True).value_counts()
df[gols_visitante].nunique(axis=1,dropna=True).value_counts()
df[colunas_hora].nunique(axis=1,dropna=True).value_counts()

df['gols_mandante_moda'] = df[gols_mandante].mode(axis=1, dropna=True).iloc[:, 0]
df['gols_visitante_moda'] = df[gols_visitante].mode(axis=1, dropna=True).iloc[:, 0]
df['horario'] = df[colunas_hora].mode(axis=1, dropna=True).iloc[:, 0]
df.drop(columns=gols_mandante+gols_visitante+colunas_hora, inplace=True)

for i in range(1,10):
    df['horario'] = df['horario'].apply(lambda x: detox.arredondar_quase_15min(x,min=i,div=15))

df.loc[2191, 'horario'] = pd.to_datetime('2009-10-19 19:00') #horario meio noite
df['horario'] = df['horario'].dt.time

for col in arenas_col:
    df[col] = df[col].apply(detox.normalizar_texto)
    df[col] = df[col].str.replace(r'\bestadio\b', '', regex=True).str.strip()
    df[col] = df[col].str.replace(r'\(\*pf\)', '', regex=True).str.strip()
    df[col] = df[col].str.replace(r'\*\(pf\)', '', regex=True)

mapeamento_estadios = detox.ler_json_as_dict('arenas_dict.json')

for arena in arenas_col:
    df[arena] = df[arena].replace(mapeamento_estadios)

df['Arena_moda'] = df[arenas_col].mode(axis=1, dropna=True).iloc[:, 0]
df.drop(columns=arenas_col, inplace=True)

df['Rodada_moda'] = df[rodada_col].mode(axis=1, dropna=True).iloc[:, 0]
df.drop(columns=rodada_col, inplace=True)


drop_list = ["mandante_Estado", "mandante_Estado_3", "Estado Mandante", "UF", "UF Mandante", "Cidade", "Turno", "Dia da Semana", "Série",
             "ano_campeonato","valor_equipe_titular_mandante","valor_equipe_titular_visitante","idade_media_titular_mandante",
             "idade_media_titular_mandante", "idade_media_titular_visitante", "gols_1_tempo_mandante", "gols_1_tempo_visitante", "escanteios_mandante", "escanteios_visitante", 
             "faltas_mandante", "faltas_visitante", "chutes_bola_parada_mandante", "chutes_bola_parada_visitante", "defesas_mandante", "defesas_visitante", "impedimentos_mandante", "impedimentos_visitante", "chutes_mandante", 
             "chutes_visitante", "chutes_fora_mandante", "chutes_fora_visitante", "ID","ID_3","Season", "HG", "AG", "PSCH", "PSCD", "PSCA", "MaxCH", "MaxCD", "MaxCA", "AvgCH", "AvgCD", "AvgCA", "BFECH", "BFECD", "BFECA", "mandante_gols_fora_casa", "mandante_empates_fora_casa",
             "mandante_total_jogos", "mandante_gols_casa", "mandante_jogos_fora_casa", "mandante_vitorias_casa", "mandante_derrotas_casa", "mandante_total_pontos", "mandante_empates_casa", "mandante_pontos_fora_casa", "mandante_total_gols_sofridos", "mandante_total_vitorias", "mandante_vitorias_fora_casa", "mandante_total_derrotas", 
             "mandante_pontos_casa", "mandante_derrotas_fora_casa", "mandante_total_gols_marcados", "mandante_jogos_casa", "mandante_total_empates", "visitante_gols_fora_casa", "visitante_empates_fora_casa", "visitante_total_jogos", "visitante_gols_casa", "visitante_jogos_fora_casa", "visitante_vitorias_casa", "visitante_derrotas_casa", 
             "visitante_total_pontos", "visitante_empates_casa", "visitante_pontos_fora_casa", "visitante_total_gols_sofridos", "visitante_total_vitorias", "visitante_vitorias_fora_casa", "visitante_total_derrotas", "Número do Jogo","Temporada","Gols no Jogo",
             "visitante_pontos_casa", "visitante_derrotas_fora_casa", "visitante_total_gols_marcados", "visitante_jogos_casa", "visitante_total_empates",
             "Placar","Link do Jogo","Data de Início da Consulta","Data de Fim da Consulta",    "UF Visitante", "Resultado", "Resultado Mandante", "Resultado Visitante", "ano campeonato",
            "tecnico_mandante", "tecnico_visitante", "colocacao_mandante", "colocacao_visitante",
            "formacao_mandante", "formacao_visitante", "tecnico_mandante_1", "tecnico_visitante_1",
            "vencedor", "visitante_Estado", "Dia", "Vencedor", "Estado Visitante", "Estado Vencedor",
            "formacao_mandante_3", "formacao_visitante_3", "tecnico_mandante_3", "tecnico_visitante_3",
            "vencedor_3", "visitante_Estado_3", "Country", "League", "Time", "Res","resultado"
]
df.drop(columns=drop_list, inplace=True)

# ajustando rodadas que ficaram erradas
df.loc[(df['Mandante'] == 'Athletico Paranaense') & (df['Visitante'] == 'Santos') & (df['edition'] == 2003), 'Rodada_moda'] = 14 
df.loc[(df['Mandante'] == 'Cruzeiro') & (df['Visitante'] == 'Botafogo') & (df['edition'] == 2007), 
       ['Rodada_moda','Data','publico','publico_max','gols_mandante_moda','gols_visitante_moda','arbitro']] = [15,pd.to_datetime('2009-7-29'),13302,19226,3,2,'Leonardo Gaciba da Silva'] 

df.loc[(df['Mandante'] == 'Botafogo') & (df['Visitante'] == 'Flamengo') & (df['edition'] == 2009), 
       ['Rodada_moda','Data','arbitro','publico','gols_mandante_moda','gols_visitante_moda']] = [31,pd.to_datetime('2009-10-25'),'Luiz Antônio Silva dos Santos',25192,0,1] 

df.loc[(df['Mandante'] == 'Flamengo') & (df['Visitante'] == 'Botafogo') & (df['edition'] == 2009), 
       ['Rodada_moda','horario']] = [12,pd.to_datetime('18:30:00', format='%H:%M:%S')] 

df.loc[(df['Mandante'] == 'Chapecoense') & (df['Visitante'] == 'Atlético Mineiro') & (df['edition'] == 2016), 'publico'] = 0 

df.to_excel(r'C:\Users\USER\Desktop\GitHub\Brasileirao-Analise\1. Data\Transformed\Feat_eng.xlsx',index=False)

Feature Engineering

In [169]:
importlib.reload(detox)

df = pd.read_excel(r'C:\Users\USER\Desktop\GitHub\Brasileirao-Analise\1. Data\Transformed\Feat_eng.xlsx')

#criar colunas de pontuação na rodada
df.rename(columns={'gols_mandante_moda': 'Placar mandante', 'gols_visitante_moda': 'Placar visitante', 'Rodada_moda': 'Rodada', 'Arena_moda': 'Arena'}, inplace=True)
df = detox.distribuir_pontos(df,'Placar mandante','Placar visitante')

df.sort_values(by=['edition','Rodada','Data'],ascending=[True, True,True], inplace=True,ignore_index=True)
season_list = list(set(df['edition']))
for ano in season_list:
    df = detox.pont_acumulada(df,ano)

df.rename(columns={"publico_max":"publico disponivel","edition":"temporada","publico":"publico pagante"},inplace=True)

map_publico_maximo = detox.ler_json_as_dict('estadios.json')
df['capacidade arena'] = df['Arena'].map(map_publico_maximo)

# correção manual publico, capacidade e disponibilidade
df.loc[(df['Mandante'] == 'Santos') & (df['Visitante'] == 'Portuguesa') & (df['temporada'] == 2008), 'publico pagante'] = 7552 

# atribuindo capacidade maxima a venda disponivel para incoerencias que a venda é maior que a capacidade
mask_capacidade = df['publico disponivel'] > df['capacidade arena']
df.loc[mask_capacidade,'publico disponivel'] = df.loc[mask_capacidade,'capacidade arena']

# criadno aproveitamento de cada mandante nas ultimas x rodadas
for ano in list(set(df['temporada'])):
    df = detox.yield_last_xrounds(df,3,ano)
    df = detox.yield_last_xrounds(df,5,ano)

df.to_excel(r'C:\Users\USER\Desktop\GitHub\Brasileirao-Analise\1. Data\Processed\data_exploratory.xlsx',index=False)